# *Aufgabe: SVM und Interpretation ML*

0. Imports

In [ ]:
import os

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

### Aufgabe 1. Gaussian Naive Bayes

Laden Sie den Census Datensatz und implementieren Sie den Gaussian Naive Bayes Algorithmus analog zu den Vorlesungsinhalten “from Scratch” (dh. ohne die GNB Implementierung aus sklearn). Sagen Sie nur auf Grundlage von Alter und Arbeitsstunden pro Woche die Zielvariable der Einkommensklasse für eine Instanz vorher. Ziehen Sie das Notebook 02/EDA heran, wenn Sie eine Gaußsche Funktion aus einer Datenmenge erstellen wollen.


In [2]:
# Laden und Preprocessing wie in vorherigen Notebooks
census = pd.read_csv(os.path.join("..", "..", "..", "data", "census.csv"))[['age', 'hours-per-week', 'target']]
census["target"] = census["target"].map({" <=50K":0," >50K":1})
census.head(5)

,age,hours-per-week,target
0,39,40,0
1,50,13,0
2,38,40,0
3,53,40,0
4,28,40,0


In [3]:
x_train, x_test, y_train, y_test = train_test_split(census.drop(["target"],axis=1),census["target"])
print('X Train: {}'.format(x_train.shape))
print('Y Train: {}'.format(y_train.shape))
print('X Test: {}'.format(x_test.shape))
print('Y Test: {}'.format(y_test.shape))

# Normalize: Age und hours-per-week
minmax = MinMaxScaler()

x_train['age'] = minmax.fit_transform(x_train['age'].to_numpy().reshape(-1,1))
x_train['hours-per-week'] = minmax.fit_transform(x_train['hours-per-week'].to_numpy().reshape(-1,1))

x_test['age'] = minmax.fit_transform(x_test['age'].to_numpy().reshape(-1,1))
x_test['hours-per-week'] = minmax.fit_transform(x_test['hours-per-week'].to_numpy().reshape(-1,1))

X Train: (24420, 2)
Y Train: (24420,)
X Test: (8141, 2)
Y Test: (8141,)


2. Gaussian Naive Bayes:

In [4]:
class GaussianNBFromScratch:
    """
    Eine "from scratch" Implementierung des Gaussian Naive Bayes Algorithmus.
    """

    def fit(self, X, y):
        """
        Lernen der Mittelwerte, Standardabweichungen und Priors aus den Trainingsdaten.
        """
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)

        # Initialisieren von mean, var und priors
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def _calculate_likelihood(self, x, mean, var):
        """
        Berechnet die Wahrscheinlichkeitsdichte mit der Gausschen Formel.
        Dies ist die Funktion, die laut Hinweis aus den Daten erstellt wird.
        """
        exponent = np.exp(-((x - mean) ** 2 / (2 * var)))
        return (1 / np.sqrt(2 * np.pi * var)) * exponent

    def predict(self, X):
        """
        Sagt die Klasse für eine oder mehrere neue Instanzen voraus.
        """
        y_pred = [self._predict_single(x) for x in X]
        return np.array(y_pred)

    def _predict_single(self, x):
        """
        Führt die Klassenvorhersage für eine einzelne Instanz durch.
        """
        posteriors = []

        # Berechne die Posterior-Wahrscheinlichkeit für jede Klasse
        for idx, c in enumerate(self.classes):
            prior = np.log(self._priors[idx])
            # Wir verwenden log-Wahrscheinlichkeiten, um numerische Instabilität zu vermeiden
            class_conditional = np.sum(
                np.log(self._calculate_likelihood(x, self._mean[idx], self._var[idx]))
            )
            posterior = prior + class_conditional
            posteriors.append(posterior)

        # Gib die Klasse mit der höchsten Posterior-Wahrscheinlichkeit zurück
        return self.classes[np.argmax(posteriors)]

In [ ]:
# Modell trainieren und Vorhersage für eine Instanz durchführen

# Vorbereiten der Features (X) und des Ziels (y)
X = census[["age", "hours-per-week"]].values
y = census["target"].values

# Initialisieren und Trainieren des Modells
gnb = GaussianNBFromScratch()
gnb.fit(X, y)

# Definieren einer neuen Instanz, für die eine Vorhersage getroffen werden soll
# Beispiel: Eine Person, 40 Jahre alt, die 45 Stunden pro Woche arbeitet.
age, hours = 40, 45
new_instance = np.array([[age, hours]])

# Vorhersage treffen
prediction = gnb.predict(new_instance)

print("\n--- Vorhersage für eine neue Instanz ---")
print(f"Daten der Instanz: Alter={age}, Arbeitsstunden/Woche={hours}")
print(f"Vorhergesagte Einkommensklasse: {prediction[0]}")



--- Vorhersage für eine neue Instanz ---
Daten der Instanz: Alter=40, Arbeitsstunden/Woche=45
Vorhergesagte Einkommensklasse: 0
